In [1]:
# Algoritam normalizacije

import numpy as np
from numpy import linalg as LA
import math

In [2]:
# definicija Tacke

class Tacka:
    # izracunamo odmah afine koordinate
    def __init__(self,ime, x, y, z):
        self.x = x / z
        self.y = y / z
        self.z = 1
        self.ime = ime
        
    def getArray(self):
        return np.array([self.x, self.y, self.z])
    
    def __add__(self, other): 
        return Tacka(self.ime + " " + other.ime,
                     self.x + other.x,
                     self.y + other.y,
                     self.z * other.z) 
    
    def __str__(self):
        return "(%f, %f, %f)" % (self.x, self.y, self.z)

In [3]:
class DLT:
    def __init__(self, nizTacaka, nizSlikaTacaka):
        self.tacke = nizTacaka
        self.slike = nizSlikaTacaka
        
    def napraviDvaRedaZaTacke(self, tacka, slika):
        t = tacka.getArray()
        tp = slika.getArray()
        
        dvaReda = np.zeros((2, 9))
        dvaReda[0][3] = t[0]*tp[2]*-1
        dvaReda[0][4] = t[1]*tp[2]*-1
        dvaReda[0][5] = t[2]*tp[2]*-1
        dvaReda[0][6] = t[0]*tp[1]
        dvaReda[0][7] = t[1]*tp[1]
        dvaReda[0][8] = t[2]*tp[1]
        
        dvaReda[1][0] = t[0]*tp[2]
        dvaReda[1][1] = t[1]*tp[2]
        dvaReda[1][2] = t[2]*tp[2]
        dvaReda[1][6] = t[0]*tp[0]*-1
        dvaReda[1][7] = t[1]*tp[0]*-1
        dvaReda[1][8] = t[2]*tp[0]*-1

        return dvaReda
        
    def izvrsi(self):
        matrica = self.napraviDvaRedaZaTacke(self.tacke[0], self.slike[0])
        print(matrica)
        # 1) i 2) u petlji dodajemo u matricu po dva reda za svaku tacku
        n = np.size(self.tacke)
        
        for i in range(1, n):
            matrica = np.vstack((matrica , self.napraviDvaRedaZaTacke(self.tacke[i], self.slike[i])))
            print("____________________")
            print(matrica)
            print("____________________")                          
            
        # 3) radimo svn dekompoziciju i pravimo matricu
        u, s, v = np.linalg.svd(matrica, full_matrices=False)
        #print(v)
        A = np.array([v[-1][:3],v[-1][3:-3],v[-1][-3:]])
        print(A)
        
        return A

In [4]:
class Normalizacija:
    def __init__(self, nizTacaka):
        self.nizTacaka = nizTacaka
        
    def rastojanjeOdKoordinatnogPocetka(self, tacka):
        return math.sqrt(tacka.x*tacka.x + tacka.y*tacka.y)
        
    def izvrsi(self):
        rezultujuciNiz = self.nizTacaka
        
        # nadji teziste 
        teziste = np.sum(self.nizTacaka) # / self.nizTacaka.size()
        
        #print("tezisna tacka: ")
        #print(teziste)
        
        n = np.size(self.nizTacaka)
        #print(n)
        
        tx = teziste.x / n 
        ty = teziste.y / n
        
        #print("tx, ty:")
        #print(tx, ty)
        
        # napravi matricu translacije koja ce slikati teziste
        # u koordniatni pocetak
        G = np.array([[1, 0, -tx], 
                      [0, 1, -ty],
                      [0, 0, 1 ]])
        
        for tacka in rezultujuciNiz:
            tacka.x = tacka.x - tx
            tacka.y = tacka.y - ty
        
        print("G:")
        print(G)
        
        # Skaliranje! 
        # k = sqrt(2) / prosecnaUdaljenost
        
        #print("tacke nakon translacije:")
        prosecnaUdaljenost = 0
        for tacka in rezultujuciNiz:
            #print(tacka)
            prosecnaUdaljenost += self.rastojanjeOdKoordinatnogPocetka(tacka)
        
        prosecnaUdaljenost /= n
        
        #print("prosecna udaljenost:")
        #print(prosecnaUdaljenost)
        
        k = math.sqrt(2) / prosecnaUdaljenost
        
        #print("k:")
        #print(k)
        
        S = np.array([[k, 0, 0],
                      [0, k, 0],
                      [0, 0, 1]])
        
        print("S:")
        print(S)
        
        T = np.dot(S, G)
        print("T=SG")
        print(T)
        
        for tacka in rezultujuciNiz:
            tacka.x *= k
            tacka.y *= k
            print(tacka)
            
        return rezultujuciNiz

In [5]:
M1 = Tacka("M1", 7, 4, 1)
M2 = Tacka("M2", 6, 4, 2)
M3 = Tacka("M3", 5, 2, 0.5)
M4 = Tacka("M4", 0, 0, 10)
M5 = Tacka("M5", -5,-5,-1)

niz = np.array([M1, M2, M3, M4, M5])

normalizacija = Normalizacija(niz)
normalizacija.izvrsi()

G:
[[ 1.  0. -5.]
 [ 0.  1. -3.]
 [ 0.  0.  1.]]
S:
[[0.406334 0.       0.      ]
 [0.       0.406334 0.      ]
 [0.       0.       1.      ]]
T=SG
[[ 0.406334    0.         -2.03166998]
 [ 0.          0.406334   -1.21900199]
 [ 0.          0.          1.        ]]
(0.812668, 0.406334, 1.000000)
(-0.812668, -0.406334, 1.000000)
(2.031670, 0.406334, 1.000000)
(-2.031670, -1.219002, 1.000000)
(0.000000, 0.812668, 1.000000)


array([<__main__.Tacka object at 0x00000188CDEC3E08>,
       <__main__.Tacka object at 0x00000188CDEC3B88>], dtype=object)

In [6]:
# DLTSaNormalizacijom

A = Tacka("A", -3, -1, 1)
B = Tacka("B", 3, -1, 1)
C = Tacka("C", 1, 1, 1)
D = Tacka("D", -1, 1, 1)
E = Tacka("E", 1,2,3)
F = Tacka("F", -8, -2, 1)

tacke = np.array([A, B, C, D, E, F])
normalizacija = Normalizacija(tacke)
tacke = normalizacija.izvrsi()

Ap = Tacka("A'", -2, -1, 1)
Bp = Tacka("B'", 2, -1, 1)
Cp = Tacka("C'", 2, 1, 1)
Dp = Tacka("D'", -2, 1, 1)
Ep = Tacka("E'", 2, 1, 4)
Fp = Tacka("F'", -16, -5, 4)

slike = np.array([Ap, Bp, Cp, Dp, Ep, Fp])
normalizacija = Normalizacija(slike)
e = normalizacija.izvrsi()

dlt = DLT(tacke, slike)
A = dlt.izvrsi()


G:
[[1.         0.         1.27777778]
 [0.         1.         0.22222222]
 [0.         0.         1.        ]]
S:
[[0.4496856 0.        0.       ]
 [0.        0.4496856 0.       ]
 [0.        0.        1.       ]]
T=SG
[[0.4496856  0.         0.57459826]
 [0.         0.4496856  0.09993013]
 [0.         0.         1.        ]]
(-0.774459, -0.349755, 1.000000)
(1.923655, -0.349755, 1.000000)
(1.024284, 0.549616, 1.000000)
(0.124913, 0.549616, 1.000000)
(0.724493, 0.399721, 1.000000)
(-3.022887, -0.799441, 1.000000)
G:
[[1.         0.         0.58333333]
 [0.         1.         0.16666667]
 [0.         0.         1.        ]]
S:
[[0.61609012 0.         0.        ]
 [0.         0.61609012 0.        ]
 [0.         0.         1.        ]]
T=SG
[[0.61609012 0.         0.3593859 ]
 [0.         0.61609012 0.10268169]
 [0.         0.         1.        ]]
(-0.872794, -0.513408, 1.000000)
(1.591566, -0.513408, 1.000000)
(1.591566, 0.718772, 1.000000)
(-0.872794, 0.718772, 1.000000)
(0.667431, 0.2